<a href="https://colab.research.google.com/github/mcleantom/NeuralNetworksFromScratch/blob/main/micrograd_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>